[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ryansteed/image-gpt-bias/blob/master/notebooks/experiments.ipynb)

# Image GPT Bias
**Image Embedding Association Test**

Ryan Steed

This script adapted from https://colab.research.google.com/github/apeguero1/image-gpt/blob/master/Transformers_Image_GPT.ipynb.


In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

%load_ext autoreload
%autoreload 2

%cd /Users/steed/caliskan/git/image-gpt-bias

/Users/steed/caliskan/git/image-gpt-bias


## Download Model

In [ ]:
!nvidia-smi #OpenAI says you need 16GB GPU for the large model, but it may work if you lower n_sub_batch on the others.

In [ ]:
import urllib

# don't share these to Github or elsewhere!
u = "your_username"
p = urllib.parse.quote("your_password")
!git clone --recurse-submodules -j8 https://$u:$p@github.com/$u/image-gpt-bias.git
%cd image-gpt-bias

In [ ]:
# note - you may need to restart the kernel for these installations to take effect
!pip install -e .
!pip install -e weat

In [2]:
model_sizes = ["s", "m", "l"] #small medium large, xl not available
model_size = "s"
models_dir = "models"
color_clusters_dir = "clusters"
n_px = 32

In [ ]:
# download the model - skip if already downloaded
!python image-gpt/download.py --model {model_size} --ckpt 1000000 --clusters --download_dir {models_dir}/{model_size}
!python image-gpt/download.py --clusters --download_dir {color_clusters_dir}

## IATs

### Quick Summary

In [25]:
from ieat.api import test_all

import pandas as pd
import numpy as np

results = test_all(
    model_types=["logit", "sent", "openai"],
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px
)

Running Insect-Flower


2020-09-18 16:54:36,264 [79688] WARNING  root:202: [JupyterRequire] Equalities contributed 1/924 to p-value


Running Weapon


2020-09-18 16:54:36,397 [79688] WARNING  root:202: [JupyterRequire] Equalities contributed 1/924 to p-value
2020-09-18 16:54:36,521 [79688] WARNING  root:202: [JupyterRequire] Equalities contributed 1/924 to p-value
2020-09-18 16:54:36,642 [79688] WARNING  root:202: [JupyterRequire] Equalities contributed 1/924 to p-value


Running Weapon (Modern)


...

Running Arab-Muslim


2020-09-18 16:54:43,436 [79688] WARNING  root:202: [JupyterRequire] Equalities contributed 1/924 to p-value


Running Age


2020-09-18 16:54:43,585 [79688] WARNING  root:202: [JupyterRequire] Equalities contributed 1/924 to p-value
2020-09-18 16:54:43,736 [79688] WARNING  root:202: [JupyterRequire] Equalities contributed 1/924 to p-value


Running Gender-Science
Running Gender-Career


In [26]:
import pandas as pd

results_df = pd.DataFrame(results).transpose()
results_df.columns = ["X", "Y", "A", "B", "effect", "p", "n_target", "n_attr"]
for c in results_df.columns[:4]:
    results_df[c] = results_df[c].str.split("/").str[-1]
results_df["sig"] = ""
for l in [0.10, 0.05, 0.01]:
    results_df.sig[results_df.p < l] += "⭐"
# results_df[results_df.index.get_level_values(1) == "openai"]
results_df

X                 Y  \
Insect-Flower                    logit              flower            insect   
                                 sent               flower            insect   
                                 openai             flower            insect   
Weapon                           logit               white             black   
                                 sent                white             black   
                                 openai              white             black   
Weapon (Modern)                  logit               white             black   
                                 sent                white             black   
                                 openai              white             black   
Native                           logit                euro            native   
                                 sent                 euro            native   
                                 openai               euro            native   
Asian                            logit   european-american    asian-american   
                                 sent    european-american    asian-american   
                                 openai  european-american    asian-american   
Weight                           logit                thin               fat   
                                 sent                 thin               fat   
                                 openai               thin               fat   
Skin-Tone                        logit               light              dark   
                                 sent                light              dark   
                                 openai              light              dark   
Disability                       logit            disabled             abled   
                                 sent             disabled             abled   
                                 openai           disabled             abled   
President - Kennedy vs. Trump    logit             kennedy             trump   
                                 sent              kennedy             trump   
                                 openai            kennedy             trump   
President - B. Clinton vs. Trump logit             clinton             trump   
                                 sent              clinton             trump   
                                 openai            clinton             trump   
President - Bush vs. Trump       logit                bush             trump   
                                 sent                 bush             trump   
                                 openai               bush             trump   
President - Lincoln vs. Trump    logit             lincoln             trump   
                                 sent              lincoln             trump   
                                 openai            lincoln             trump   
Religion                         logit        christianity           judaism   
                                 sent         christianity           judaism   
                                 openai       christianity           judaism   
Sexuality                        logit                 gay          straight   
                                 sent                  gay          straight   
                                 openai                gay          straight   
Race                             logit   european-american  african-american   
                                 sent    european-american  african-american   
                                 openai  european-american  african-american   
Arab-Muslim                      logit        other-people       arab-muslim   
                                 sent         other-people       arab-muslim   
                                 openai       other-people       arab-muslim   
Age                              logit               young               old   
                                 sent                youn

### Detail View

#### Weapons

In [ ]:
from ieat.api import test

test(
    "data/experiments/weapon/black",
    "data/experiments/weapon/white",
    "data/experiments/weapon/tool",
    "data/experiments/weapon/weapon",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px
#     from_cache=False
)

#### Native

In [ ]:
test(
    "data/experiments/native/euro",
    "data/experiments/native/native",
    "data/experiments/native/us",
    "data/experiments/native/world",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px
#     from_cache=False
)

#### Asian

In [ ]:
test(
    "data/experiments/asian/european-american",
    "data/experiments/asian/asian-american",
    "data/experiments/asian/american",
    "data/experiments/asian/foreign",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px
#     from_cache=False
)

#### Insect-Flower

In [ ]:
from ieat.api import test

test(
    "data/experiments/insect-flower/flower",
    "data/experiments/insect-flower/insect",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Weight

In [ ]:
from ieat.api import test

test(
    "data/experiments/weight/thin",
    "data/experiments/weight/fat",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Skin-Tone

In [ ]:
from ieat.api import test

test(
    "data/experiments/skin-tone/light",
    "data/experiments/skin-tone/dark",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Disability

In [ ]:
from ieat.api import test

test(
    "data/experiments/disabled/disabled",
    "data/experiments/disabled/abled",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Presidents

In [ ]:
from ieat.api import test

test(
    "data/experiments/presidents/trump",
    "data/experiments/presidents/kennedy",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

In [ ]:
from ieat.api import test

test(
    "data/experiments/presidents/trump",
    "data/experiments/presidents/clinton",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

In [ ]:
from ieat.api import test

test(
    "data/experiments/presidents/trump",
    "data/experiments/presidents/bush",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

In [ ]:
from ieat.api import test

test(
    "data/experiments/presidents/trump",
    "data/experiments/presidents/lincoln",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Religion

In [ ]:
from ieat.api import test

test(
    "data/experiments/religion/christianity",
    "data/experiments/religion/judaism",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Gender-Science

In [ ]:
from ieat.api import test

test(
    "data/experiments/gender/science",
    "data/experiments/gender/liberal-arts",
    "data/experiments/gender/male",
    "data/experiments/gender/female",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Gender-Career

In [ ]:
from ieat.api import test

test(
    "data/experiments/gender/career",
    "data/experiments/gender/family",
    "data/experiments/gender/male",
    "data/experiments/gender/female",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Sexuality

In [ ]:
from ieat.api import test

test(
    "data/experiments/sexuality/gay",
    "data/experiments/sexuality/straight",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Race

In [ ]:
from ieat.api import test

test(
    "data/experiments/race/african-american",
    "data/experiments/race/european-american",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Arab-Muslim

In [ ]:
from ieat.api import test

test(
    "data/experiments/arab-muslim/other-people",
    "data/experiments/arab-muslim/arab-muslim",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Age

In [ ]:
from ieat.api import test

test(
    "data/experiments/age/young",
    "data/experiments/age/old",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

### Download Cached Embeddings

In [ ]:
# to download from colab
from google.colab import files

!zip -r embeddings_colab.zip embeddings
files.download("embeddings_colab.zip")